# Semantic Segmentation-KNet

## Enter the MMSegmentation home directory

In [ ]:
import os
os.chdir('mmsegmentation')

In [13]:
os.getcwd()

'/home/featurize/work/CFA/mmsegmentation'

## Load the model config file

In [14]:
from mmengine import Config
cfg = Config.fromfile('./configs/knet/knet-s3_swin-l_upernet_8xb2-adamw-80k_ade20k-512x512.py')
dataset_cfg = Config.fromfile('./configs/_base_/datasets/ZihaoDataset_pipeline.py')
cfg.merge_from_dict(dataset_cfg)

## Modify the config file

In [15]:
# Number of classes
NUM_CLASS = 4

In [16]:
# For single GPU training, replace SyncBN with BN
cfg.norm_cfg = dict(type='BN', requires_grad=True) # Use BN instead of SyncBN when using only GPU
cfg.model.data_preprocessor.size = cfg.crop_size

# Set the number of classes for model decode/auxiliary output heads
# cfg.model.decode_head.num_classes = NUM_CLASS
cfg.model.decode_head.kernel_generate_head.num_classes = NUM_CLASS
cfg.model.auxiliary_head.num_classes = NUM_CLASS

# Training Batch Size
cfg.train_dataloader.batch_size = 4

# Directory to save results
cfg.work_dir = './work_dirs/ZihaoDataset-KNet'

cfg.train_cfg.max_iters = 20000 # Number of training iterations
cfg.train_cfg.val_interval = 500 # Model evaluation interval
cfg.default_hooks.logger.interval = 100 # Logging interval
cfg.default_hooks.checkpoint.interval = 2500 # Model checkpoint saving interval
cfg.default_hooks.checkpoint.max_keep_ckpts = 2 # Maximum number of checkpoints to keep
cfg.default_hooks.checkpoint.save_best = 'mIoU' # Save the model with the highest metric

# Random seed
cfg['randomness'] = dict(seed=0)

## View the full config configuration file

In [17]:
print(cfg.pretty_text)

checkpoint_file = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/swin/swin_large_patch4_window7_224_22k_20220308-d5bdebaf.pth'
conv_kernel_size = 1
crop_size = (
    512,
    512,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        512,
        512,
    ),
    std=[
        58.395,
        57.12,
        57.375,
    ],
    type='SegDataPreProcessor')
data_root = 'CFA87_Semantic_Seg_Labelme/'
dataset_type = 'ZihaoDataset'
default_hooks = dict(
    checkpoint=dict(
        by_epoch=False,
        interval=2500,
        max_keep_ckpts=2,
        save_best='mIoU',
        type='CheckpointHook'),
    logger=dict(interval=100, log_metric_by_epoch=False, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='SegVisualizationHook

## Save the final config configuration file

In [18]:
cfg.dump('Zihao-Configs/ZihaoDataset_KNet_20230818.py')